In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import asyncio
import os

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
open_router_key = os.getenv("OPEN_ROUTER_API_KEY")

In [3]:
model_client = OpenAIChatCompletionClient(
    model="gemini-1.5-flash",
    api_key=api_key,
)

In [5]:
planning_agent = AssistantAgent(
    name="PlanningAgent",
    model_client=model_client,
    description="An agent that plans tasks, This agent should be first engage when a new task is defined.",
    system_message="""
    You are a planning agent and you are first to execute no matter what when a task comes and plan it. 
    Your role is to break down complex tasks into smaller and managable subtasks.
    Your Team Members are:
    - WebSearhAgent: Conducts web searches to gather information.
    - DataAnalystAgent: do calculations.

    Your only plan and delicate tasks - you do not execute them yourself.
    When Assigning tasks, use the following format:
    1. <agent_name>: <task>

    After the task is completed, and also in planning after everything is done tell the last agent which is going the perform the final task and gte answer to end the conversation with "TERMINATE".
    """,
)


In [6]:
def web_search_tool(query: str) -> str:
    """A mock web search tool that simulates a web search."""
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."


web_search_agent = AssistantAgent(
    name="WebSearchAgent",
    model_client=model_client,
    description="An agent that conducts web searches to gather information.",
    tools=[web_search_tool],
    system_message="""
    You are a web search agent and your only job is to conduct web searches to gather information.
    Your only tool is web_search_tool, which simulates a web search.
    Use the web_search_tool function to simulate a web search.
    You make only one web search per query.
    Your role is to conduct web searches to gather information. 
    You will be provided with queries, and you should return the relevant information.
    Once you have the information, You don't or never do data analysis or planing in the team.
    """, 
)

In [7]:
def percentage_change_tool(start: float, end: float) -> float:
    """Calculate the percentage change from old_value to new_value."""
    if start == 0:
        return float('inf') 
    return ((end - start) / start) * 100
   


data_analyst_agent = AssistantAgent(
    name="DataAnalystAgent",
    model_client=model_client,
    description="An agent that analyzes data and provides insights.",
    tools=[percentage_change_tool],
    system_message="""
    You are a data analyst agent and your only job is to analyze data based only the tools given to you you should only perform when you get an task.
    Your role is to analyze data and analyse the data and provide results using the tools provided.
    You will be provided with data, and you should return the relevant insights.
    You do not conduct web searches or planning in the team.

    if you not seen the data you should ask to provide you with the data.
    """,
)

In [8]:
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
text_mention_termination = TextMentionTermination("TERMINATE")
max_message_termination = MaxMessageTermination(10)
termination_condition = text_mention_termination | max_message_termination

In [9]:
selector_prompt = """
Select an agent to perform an task.
And Always as the planning agent as first to assign the task first.
and then the selected agent will perform the task.


{roles}

current conversation history:
{history}

Read the conversation history and then select the most appropriate agent from {participants} to perform the task.
Make sure the planner agent has assigned task before other agents start working.
Only select one agent to perform the task.
"""

In [10]:
from autogen_agentchat.teams import SelectorGroupChat
team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    selector_prompt=selector_prompt,
    termination_condition=termination_condition,
    allow_repeated_speaker=True
)

In [ ]:
from autogen_agentchat.ui import Console
task = "Who was the top scorer for Miami Heat in the 2006-2007 season? What was the total points scored by each player? and what was the percentage change in points scored by Dwayne Wade from 2006-2007 to 2007-2008 season?"

await Console(team.run_stream(task=task))

## Custom Selector Agent

In [12]:
from typing import Sequence
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
def selector_func(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
    if(messages[-1].source != planning_agent.name):
        return planning_agent.name
    return None

In [ ]:
await team.reset()

In [ ]:
from autogen_agentchat.teams import SelectorGroupChat
team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    selector_prompt=selector_prompt,
    termination_condition=termination_condition,
    allow_repeated_speaker=True,
    selector_func=selector_func
)

In [ ]:
from autogen_agentchat.ui import Console
task = "Who was the top scorer for Miami Heat in the 2006-2007 season?what was the percentage change in total between 2006-2007 to 2007-2008 seasons?"

await Console(team.run_stream(task=task))